In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 27.4 MB/s eta 0:00:00


In [ ]:
# 필요 패키지
import pandas as pd
import numpy as np
import urllib.request

import matplotlib.pyplot as plt

# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x7dac5ce90d30>)

In [ ]:
df_train = pd.read_table('ratings_train.txt')

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
df_train['document'] = df_train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "").str.strip()
df_train.head()

<ipython-input-4-ac06aefdaca7>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['document'] = df_train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "").str.strip()


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
df_train.loc[df_train['document'] == '', 'document'].value_counts()

    1255
Name: document, dtype: int64

In [ ]:
df_train['document'].replace("", np.nan, inplace=True)
df_train = df_train.dropna(how='any')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148740 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        148740 non-null  int64 
 1   document  148740 non-null  object
 2   label     148740 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

okt = Okt()

tfidf_vect = TfidfVectorizer(tokenizer=okt.morphs)
tfidf_vect.fit(df_train['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7dac562cb6d0>>)

In [ ]:
X_train_tfidf_vect = tfidf_vect.transform(df_train['document'])
y_train = df_train['label']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()

lr_clf.fit(X_train_tfidf_vect, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from joblib import dump, load

dump(lr_clf, 'korean_model.pkl')

['korean_model.pkl']

In [ ]:
dump(tfidf_vect, 'tfidf_vect.pkl')

['tfidf_vect.pkl']